## Importing packages

In [2]:
import sys, os
import pickle as pkl

import numpy as np

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

C:\Users\user\anaconda3\lib\site-packages\xgboost\compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\user\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\user\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\user\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.UInt64Index is deprecated and will be removed fr

In this notebook, cross-validation for ML deisotoping models is performed. 

Open Datasets 

<b> !! They are saved in */data/datasets* !! </b>

In [5]:
# Open train dataset
path = os.path.join(os.path.dirname(os.path.abspath('..')), 'data', 'datasets', 'full_train_df.pkl')
with open(path, 'rb') as f:
    dataset = pkl.load(f)
    X = dataset[0]
    targets = dataset[1]

In [6]:
# Open test dataset
path = os.path.join(os.path.dirname(os.path.abspath('..')), 'data', 'datasets', 'full_test_df.pkl')
with open(path, 'rb') as f:
    test_dataset = pkl.load(f)
    test_X = test_dataset[0]
    test_targets = test_dataset[1]

Cross validation

In [4]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=3, scoring_fit='roc_auc',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

Results

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size=0.3, random_state=17)

param_grid_randomforest = {
    'n_estimators': [50, 100, 300, 500, 700, 900],
    'max_depth': [4, 6, 8, 12]
}
param_grid_boosting = {
    'n_estimators': [300, 500, 1000, 1500],
    'max_depth': [6, 8, 10, 12]
}

# Random Forest GridSearchCV
model = RandomForestClassifier(max_features=None)
model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                  param_grid_randomforest)

print("Лучший показатель в Random Forest ", model.best_score_)
print("Оптимальные гиперпараметры: ", model.best_params_)

# XGBoost GridSearchCV
model = XGBClassifier()
model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                 param_grid_boosting)

print("Лучший показатель в Xgboost ", model.best_score_)
print("Оптимальные гиперпараметры: ", model.best_params_)

# CatBoost GridSearchCV
model = CatBoostClassifier()
model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                 param_grid_boosting)

print("Лучший показатель в Catboost ", model.best_score_)
print("Оптимальные гиперпараметры: ", model.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  8.3min finished


Лучший показатель в Random Forest  0.9993140689561386
Оптимальные гиперпараметры:  {'max_depth': 12, 'n_estimators': 700}
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  4.1min finished


Лучший показатель в Xgboost  0.999639766289421
Оптимальные гиперпараметры:  {'max_depth': 10, 'n_estimators': 300}
Fitting 3 folds for each of 16 candidates, totalling 48 fits


...

795:	learn: 0.0062459	total: 45.2s	remaining: 39.9s
796:	learn: 0.0062459	total: 45.2s	remaining: 39.9s
797:	learn: 0.0062459	total: 45.3s	remaining: 39.8s
798:	learn: 0.0062459	total: 45.3s	remaining: 39.8s
799:	learn: 0.0062459	total: 45.4s	remaining: 39.7s
800:	learn: 0.0062459	total: 45.4s	remaining: 39.6s
801:	learn: 0.0062459	total: 45.5s	remaining: 39.6s
802:	learn: 0.0062459	total: 45.5s	remaining: 39.5s
803:	learn: 0.0062458	total: 45.6s	remaining: 39.4s
804:	learn: 0.0062458	total: 45.6s	remaining: 39.4s
805:	learn: 0.0062458	total: 45.7s	remaining: 39.3s
806:	learn: 0.0062458	total: 45.7s	remaining: 39.3s
807:	learn: 0.0062458	total: 45.8s	remaining: 39.2s
808:	learn: 0.0062458	total: 45.8s	remaining: 39.1s
809:	learn: 0.0062458	total: 45.9s	remaining: 39.1s
810:	learn: 0.0062458	total: 45.9s	remaining: 39s
811:	learn: 0.0062458	total: 46s	remaining: 38.9s
812:	learn: 0.0062458	total: 46s	remaining: 38.9s
813:	learn: 0.0062458	total: 46.1s	remaining: 38.8s
814:	learn: 0.0062

956:	learn: 0.0062454	total: 53.6s	remaining: 30.4s
957:	learn: 0.0062454	total: 53.6s	remaining: 30.3s
958:	learn: 0.0062453	total: 53.7s	remaining: 30.3s
959:	learn: 0.0062453	total: 53.7s	remaining: 30.2s
960:	learn: 0.0062453	total: 53.8s	remaining: 30.2s
961:	learn: 0.0062453	total: 53.8s	remaining: 30.1s
962:	learn: 0.0062453	total: 53.9s	remaining: 30s
963:	learn: 0.0062453	total: 53.9s	remaining: 30s
964:	learn: 0.0062453	total: 54s	remaining: 29.9s
965:	learn: 0.0062453	total: 54.1s	remaining: 29.9s
966:	learn: 0.0062453	total: 54.1s	remaining: 29.8s
967:	learn: 0.0062453	total: 54.2s	remaining: 29.8s
968:	learn: 0.0062453	total: 54.2s	remaining: 29.7s
969:	learn: 0.0062452	total: 54.3s	remaining: 29.6s
970:	learn: 0.0062452	total: 54.3s	remaining: 29.6s
971:	learn: 0.0062452	total: 54.4s	remaining: 29.5s
972:	learn: 0.0062452	total: 54.4s	remaining: 29.5s
973:	learn: 0.0062452	total: 54.5s	remaining: 29.4s
974:	learn: 0.0062452	total: 54.5s	remaining: 29.4s
975:	learn: 0.0062

1115:	learn: 0.0062332	total: 1m 1s	remaining: 21.3s
1116:	learn: 0.0062332	total: 1m 2s	remaining: 21.3s
1117:	learn: 0.0062332	total: 1m 2s	remaining: 21.2s
1118:	learn: 0.0062332	total: 1m 2s	remaining: 21.2s
1119:	learn: 0.0062332	total: 1m 2s	remaining: 21.1s
1120:	learn: 0.0062332	total: 1m 2s	remaining: 21s
1121:	learn: 0.0062331	total: 1m 2s	remaining: 21s
1122:	learn: 0.0062331	total: 1m 2s	remaining: 20.9s
1123:	learn: 0.0062331	total: 1m 2s	remaining: 20.9s
1124:	learn: 0.0062331	total: 1m 2s	remaining: 20.8s
1125:	learn: 0.0062331	total: 1m 2s	remaining: 20.8s
1126:	learn: 0.0062331	total: 1m 2s	remaining: 20.7s
1127:	learn: 0.0062331	total: 1m 2s	remaining: 20.6s
1128:	learn: 0.0062331	total: 1m 2s	remaining: 20.6s
1129:	learn: 0.0062331	total: 1m 2s	remaining: 20.5s
1130:	learn: 0.0062331	total: 1m 2s	remaining: 20.5s
1131:	learn: 0.0062331	total: 1m 2s	remaining: 20.4s
1132:	learn: 0.0062331	total: 1m 2s	remaining: 20.4s
1133:	learn: 0.0062331	total: 1m 2s	remaining: 20.

1273:	learn: 0.0061148	total: 1m 10s	remaining: 12.5s
1274:	learn: 0.0061148	total: 1m 10s	remaining: 12.4s
1275:	learn: 0.0061148	total: 1m 10s	remaining: 12.4s
1276:	learn: 0.0061148	total: 1m 10s	remaining: 12.3s
1277:	learn: 0.0061148	total: 1m 10s	remaining: 12.3s
1278:	learn: 0.0061148	total: 1m 10s	remaining: 12.2s
1279:	learn: 0.0061148	total: 1m 10s	remaining: 12.2s
1280:	learn: 0.0061148	total: 1m 10s	remaining: 12.1s
1281:	learn: 0.0061148	total: 1m 10s	remaining: 12.1s
1282:	learn: 0.0061147	total: 1m 10s	remaining: 12s
1283:	learn: 0.0061147	total: 1m 11s	remaining: 11.9s
1284:	learn: 0.0061094	total: 1m 11s	remaining: 11.9s
1285:	learn: 0.0061094	total: 1m 11s	remaining: 11.8s
1286:	learn: 0.0061094	total: 1m 11s	remaining: 11.8s
1287:	learn: 0.0061094	total: 1m 11s	remaining: 11.7s
1288:	learn: 0.0061029	total: 1m 11s	remaining: 11.7s
1289:	learn: 0.0061029	total: 1m 11s	remaining: 11.6s
1290:	learn: 0.0061029	total: 1m 11s	remaining: 11.6s
1291:	learn: 0.0061029	total: 

1428:	learn: 0.0059131	total: 1m 18s	remaining: 3.91s
1429:	learn: 0.0059131	total: 1m 18s	remaining: 3.86s
1430:	learn: 0.0059131	total: 1m 18s	remaining: 3.8s
1431:	learn: 0.0059131	total: 1m 18s	remaining: 3.75s
1432:	learn: 0.0059131	total: 1m 18s	remaining: 3.69s
1433:	learn: 0.0059131	total: 1m 18s	remaining: 3.63s
1434:	learn: 0.0059131	total: 1m 19s	remaining: 3.58s
1435:	learn: 0.0059131	total: 1m 19s	remaining: 3.52s
1436:	learn: 0.0059131	total: 1m 19s	remaining: 3.47s
1437:	learn: 0.0059131	total: 1m 19s	remaining: 3.41s
1438:	learn: 0.0059131	total: 1m 19s	remaining: 3.36s
1439:	learn: 0.0059131	total: 1m 19s	remaining: 3.3s
1440:	learn: 0.0059131	total: 1m 19s	remaining: 3.25s
1441:	learn: 0.0059131	total: 1m 19s	remaining: 3.19s
1442:	learn: 0.0059131	total: 1m 19s	remaining: 3.14s
1443:	learn: 0.0059131	total: 1m 19s	remaining: 3.08s
1444:	learn: 0.0059131	total: 1m 19s	remaining: 3.03s
1445:	learn: 0.0059131	total: 1m 19s	remaining: 2.97s
1446:	learn: 0.0059131	total: 

In [6]:
state_params_rf = {
        'max_features': None,
        'n_estimators': 700,
        'max_depth': 12
    }
state_params_xgb = {
        'n_estimators': 300,
        'max_depth': 10
    }
state_params_cb = {
        'n_estimators' : 1500,
        'max_depth': 10
    }
# Random Forest
clf_rf = RandomForestClassifier(**state_params_rf).fit(X, targets)

# XGBoost
clf_xgb = XGBClassifier(**state_params_xgb).fit(X, targets)

# CatBoost
clf_cb = CatBoostClassifier(**state_params_cb).fit(X, targets)

Learning rate set to 0.039923
0:	learn: 0.5994859	total: 55.2ms	remaining: 1m 22s
1:	learn: 0.5158440	total: 108ms	remaining: 1m 20s
2:	learn: 0.4374117	total: 160ms	remaining: 1m 19s
3:	learn: 0.3771129	total: 222ms	remaining: 1m 23s
4:	learn: 0.3277726	total: 278ms	remaining: 1m 23s
5:	learn: 0.2829585	total: 332ms	remaining: 1m 22s
6:	learn: 0.2400418	total: 385ms	remaining: 1m 22s
7:	learn: 0.2112465	total: 443ms	remaining: 1m 22s
8:	learn: 0.1866274	total: 497ms	remaining: 1m 22s
9:	learn: 0.1662513	total: 550ms	remaining: 1m 21s
10:	learn: 0.1455077	total: 602ms	remaining: 1m 21s
11:	learn: 0.1285295	total: 662ms	remaining: 1m 22s
12:	learn: 0.1140678	total: 715ms	remaining: 1m 21s
13:	learn: 0.1026669	total: 769ms	remaining: 1m 21s
14:	learn: 0.0931936	total: 821ms	remaining: 1m 21s
15:	learn: 0.0843567	total: 878ms	remaining: 1m 21s
16:	learn: 0.0775460	total: 934ms	remaining: 1m 21s
17:	learn: 0.0700547	total: 986ms	remaining: 1m 21s
18:	learn: 0.0644999	total: 1.04s	remaining

157:	learn: 0.0113005	total: 9.03s	remaining: 1m 16s
158:	learn: 0.0112445	total: 9.09s	remaining: 1m 16s
159:	learn: 0.0112108	total: 9.15s	remaining: 1m 16s
160:	learn: 0.0111701	total: 9.2s	remaining: 1m 16s
161:	learn: 0.0111326	total: 9.26s	remaining: 1m 16s
162:	learn: 0.0110529	total: 9.32s	remaining: 1m 16s
163:	learn: 0.0110135	total: 9.38s	remaining: 1m 16s
164:	learn: 0.0109587	total: 9.44s	remaining: 1m 16s
165:	learn: 0.0108960	total: 9.5s	remaining: 1m 16s
166:	learn: 0.0108303	total: 9.56s	remaining: 1m 16s
167:	learn: 0.0107972	total: 9.62s	remaining: 1m 16s
168:	learn: 0.0107646	total: 9.67s	remaining: 1m 16s
169:	learn: 0.0107363	total: 9.73s	remaining: 1m 16s
170:	learn: 0.0106950	total: 9.79s	remaining: 1m 16s
171:	learn: 0.0106777	total: 9.85s	remaining: 1m 16s
172:	learn: 0.0106401	total: 9.9s	remaining: 1m 15s
173:	learn: 0.0106231	total: 9.97s	remaining: 1m 16s
174:	learn: 0.0105838	total: 10s	remaining: 1m 15s
175:	learn: 0.0105335	total: 10.1s	remaining: 1m 15

315:	learn: 0.0074473	total: 18.5s	remaining: 1m 9s
316:	learn: 0.0074297	total: 18.5s	remaining: 1m 9s
317:	learn: 0.0073989	total: 18.6s	remaining: 1m 9s
318:	learn: 0.0073786	total: 18.6s	remaining: 1m 8s
319:	learn: 0.0073675	total: 18.7s	remaining: 1m 8s
320:	learn: 0.0073675	total: 18.7s	remaining: 1m 8s
321:	learn: 0.0073385	total: 18.8s	remaining: 1m 8s
322:	learn: 0.0073285	total: 18.9s	remaining: 1m 8s
323:	learn: 0.0073114	total: 18.9s	remaining: 1m 8s
324:	learn: 0.0072954	total: 19s	remaining: 1m 8s
325:	learn: 0.0072828	total: 19s	remaining: 1m 8s
326:	learn: 0.0072710	total: 19.1s	remaining: 1m 8s
327:	learn: 0.0072581	total: 19.2s	remaining: 1m 8s
328:	learn: 0.0072453	total: 19.2s	remaining: 1m 8s
329:	learn: 0.0072453	total: 19.3s	remaining: 1m 8s
330:	learn: 0.0072377	total: 19.3s	remaining: 1m 8s
331:	learn: 0.0072312	total: 19.4s	remaining: 1m 8s
332:	learn: 0.0072210	total: 19.5s	remaining: 1m 8s
333:	learn: 0.0072053	total: 19.5s	remaining: 1m 8s
334:	learn: 0.00

476:	learn: 0.0066536	total: 26.6s	remaining: 57s
477:	learn: 0.0066536	total: 26.6s	remaining: 56.9s
478:	learn: 0.0066536	total: 26.7s	remaining: 56.8s
479:	learn: 0.0066536	total: 26.7s	remaining: 56.8s
480:	learn: 0.0066535	total: 26.8s	remaining: 56.7s
481:	learn: 0.0066535	total: 26.8s	remaining: 56.6s
482:	learn: 0.0066535	total: 26.9s	remaining: 56.6s
483:	learn: 0.0066535	total: 26.9s	remaining: 56.5s
484:	learn: 0.0066534	total: 27s	remaining: 56.5s
485:	learn: 0.0066534	total: 27s	remaining: 56.4s
486:	learn: 0.0066534	total: 27.1s	remaining: 56.3s
487:	learn: 0.0066534	total: 27.1s	remaining: 56.3s
488:	learn: 0.0066534	total: 27.2s	remaining: 56.2s
489:	learn: 0.0066534	total: 27.2s	remaining: 56.1s
490:	learn: 0.0066534	total: 27.3s	remaining: 56.1s
491:	learn: 0.0066534	total: 27.3s	remaining: 56s
492:	learn: 0.0066534	total: 27.4s	remaining: 55.9s
493:	learn: 0.0066534	total: 27.4s	remaining: 55.9s
494:	learn: 0.0066534	total: 27.5s	remaining: 55.8s
495:	learn: 0.006653

636:	learn: 0.0066422	total: 34.5s	remaining: 46.7s
637:	learn: 0.0066414	total: 34.5s	remaining: 46.7s
638:	learn: 0.0066414	total: 34.6s	remaining: 46.6s
639:	learn: 0.0066414	total: 34.6s	remaining: 46.5s
640:	learn: 0.0066414	total: 34.7s	remaining: 46.5s
641:	learn: 0.0066393	total: 34.7s	remaining: 46.4s
642:	learn: 0.0066393	total: 34.8s	remaining: 46.4s
643:	learn: 0.0066373	total: 34.8s	remaining: 46.3s
644:	learn: 0.0066373	total: 34.9s	remaining: 46.3s
645:	learn: 0.0066369	total: 34.9s	remaining: 46.2s
646:	learn: 0.0066368	total: 35s	remaining: 46.1s
647:	learn: 0.0066368	total: 35s	remaining: 46.1s
648:	learn: 0.0066358	total: 35.1s	remaining: 46s
649:	learn: 0.0066311	total: 35.1s	remaining: 46s
650:	learn: 0.0066298	total: 35.2s	remaining: 45.9s
651:	learn: 0.0066298	total: 35.2s	remaining: 45.8s
652:	learn: 0.0066284	total: 35.3s	remaining: 45.8s
653:	learn: 0.0066284	total: 35.3s	remaining: 45.7s
654:	learn: 0.0066266	total: 35.4s	remaining: 45.7s
655:	learn: 0.006611

798:	learn: 0.0064803	total: 42.7s	remaining: 37.5s
799:	learn: 0.0064803	total: 42.8s	remaining: 37.4s
800:	learn: 0.0064803	total: 42.8s	remaining: 37.4s
801:	learn: 0.0064803	total: 42.9s	remaining: 37.3s
802:	learn: 0.0064803	total: 42.9s	remaining: 37.3s
803:	learn: 0.0064803	total: 43s	remaining: 37.2s
804:	learn: 0.0064803	total: 43s	remaining: 37.1s
805:	learn: 0.0064803	total: 43.1s	remaining: 37.1s
806:	learn: 0.0064803	total: 43.1s	remaining: 37s
807:	learn: 0.0064803	total: 43.1s	remaining: 36.9s
808:	learn: 0.0064803	total: 43.2s	remaining: 36.9s
809:	learn: 0.0064803	total: 43.2s	remaining: 36.8s
810:	learn: 0.0064803	total: 43.3s	remaining: 36.8s
811:	learn: 0.0064803	total: 43.3s	remaining: 36.7s
812:	learn: 0.0064803	total: 43.4s	remaining: 36.6s
813:	learn: 0.0064803	total: 43.4s	remaining: 36.6s
814:	learn: 0.0064803	total: 43.5s	remaining: 36.5s
815:	learn: 0.0064803	total: 43.5s	remaining: 36.5s
816:	learn: 0.0064803	total: 43.5s	remaining: 36.4s
817:	learn: 0.0064

957:	learn: 0.0064803	total: 50.4s	remaining: 28.5s
958:	learn: 0.0064803	total: 50.4s	remaining: 28.4s
959:	learn: 0.0064803	total: 50.5s	remaining: 28.4s
960:	learn: 0.0064803	total: 50.5s	remaining: 28.3s
961:	learn: 0.0064803	total: 50.6s	remaining: 28.3s
962:	learn: 0.0064803	total: 50.6s	remaining: 28.2s
963:	learn: 0.0064803	total: 50.7s	remaining: 28.2s
964:	learn: 0.0064803	total: 50.7s	remaining: 28.1s
965:	learn: 0.0064803	total: 50.8s	remaining: 28.1s
966:	learn: 0.0064803	total: 50.8s	remaining: 28s
967:	learn: 0.0064803	total: 50.9s	remaining: 28s
968:	learn: 0.0064803	total: 50.9s	remaining: 27.9s
969:	learn: 0.0064803	total: 51s	remaining: 27.8s
970:	learn: 0.0064803	total: 51s	remaining: 27.8s
971:	learn: 0.0064803	total: 51.1s	remaining: 27.7s
972:	learn: 0.0064803	total: 51.1s	remaining: 27.7s
973:	learn: 0.0064803	total: 51.2s	remaining: 27.6s
974:	learn: 0.0064803	total: 51.2s	remaining: 27.6s
975:	learn: 0.0064803	total: 51.3s	remaining: 27.5s
976:	learn: 0.006480

1114:	learn: 0.0064803	total: 57.8s	remaining: 20s
1115:	learn: 0.0064803	total: 57.9s	remaining: 19.9s
1116:	learn: 0.0064803	total: 57.9s	remaining: 19.9s
1117:	learn: 0.0064803	total: 57.9s	remaining: 19.8s
1118:	learn: 0.0064803	total: 58s	remaining: 19.7s
1119:	learn: 0.0064803	total: 58s	remaining: 19.7s
1120:	learn: 0.0064803	total: 58.1s	remaining: 19.6s
1121:	learn: 0.0064803	total: 58.1s	remaining: 19.6s
1122:	learn: 0.0064803	total: 58.2s	remaining: 19.5s
1123:	learn: 0.0064803	total: 58.2s	remaining: 19.5s
1124:	learn: 0.0064803	total: 58.3s	remaining: 19.4s
1125:	learn: 0.0064803	total: 58.3s	remaining: 19.4s
1126:	learn: 0.0064803	total: 58.4s	remaining: 19.3s
1127:	learn: 0.0064803	total: 58.4s	remaining: 19.3s
1128:	learn: 0.0064803	total: 58.5s	remaining: 19.2s
1129:	learn: 0.0064803	total: 58.5s	remaining: 19.2s
1130:	learn: 0.0064803	total: 58.6s	remaining: 19.1s
1131:	learn: 0.0064803	total: 58.6s	remaining: 19.1s
1132:	learn: 0.0064803	total: 58.6s	remaining: 19s
1

1272:	learn: 0.0064392	total: 1m 5s	remaining: 11.6s
1273:	learn: 0.0064392	total: 1m 5s	remaining: 11.6s
1274:	learn: 0.0064392	total: 1m 5s	remaining: 11.5s
1275:	learn: 0.0064392	total: 1m 5s	remaining: 11.5s
1276:	learn: 0.0064392	total: 1m 5s	remaining: 11.4s
1277:	learn: 0.0064392	total: 1m 5s	remaining: 11.4s
1278:	learn: 0.0064392	total: 1m 5s	remaining: 11.3s
1279:	learn: 0.0064392	total: 1m 5s	remaining: 11.3s
1280:	learn: 0.0064392	total: 1m 5s	remaining: 11.2s
1281:	learn: 0.0064392	total: 1m 5s	remaining: 11.2s
1282:	learn: 0.0064392	total: 1m 5s	remaining: 11.1s
1283:	learn: 0.0064392	total: 1m 5s	remaining: 11.1s
1284:	learn: 0.0064392	total: 1m 5s	remaining: 11s
1285:	learn: 0.0064392	total: 1m 5s	remaining: 11s
1286:	learn: 0.0064392	total: 1m 5s	remaining: 10.9s
1287:	learn: 0.0064392	total: 1m 5s	remaining: 10.9s
1288:	learn: 0.0064392	total: 1m 6s	remaining: 10.8s
1289:	learn: 0.0064392	total: 1m 6s	remaining: 10.8s
1290:	learn: 0.0064392	total: 1m 6s	remaining: 10.

1428:	learn: 0.0064391	total: 1m 12s	remaining: 3.61s
1429:	learn: 0.0064391	total: 1m 12s	remaining: 3.56s
1430:	learn: 0.0064391	total: 1m 12s	remaining: 3.51s
1431:	learn: 0.0064391	total: 1m 12s	remaining: 3.46s
1432:	learn: 0.0064391	total: 1m 12s	remaining: 3.41s
1433:	learn: 0.0064391	total: 1m 12s	remaining: 3.36s
1434:	learn: 0.0064391	total: 1m 13s	remaining: 3.31s
1435:	learn: 0.0064391	total: 1m 13s	remaining: 3.25s
1436:	learn: 0.0064391	total: 1m 13s	remaining: 3.2s
1437:	learn: 0.0064391	total: 1m 13s	remaining: 3.15s
1438:	learn: 0.0064391	total: 1m 13s	remaining: 3.1s
1439:	learn: 0.0064391	total: 1m 13s	remaining: 3.05s
1440:	learn: 0.0064391	total: 1m 13s	remaining: 3s
1441:	learn: 0.0064391	total: 1m 13s	remaining: 2.95s
1442:	learn: 0.0064391	total: 1m 13s	remaining: 2.9s
1443:	learn: 0.0064391	total: 1m 13s	remaining: 2.85s
1444:	learn: 0.0064391	total: 1m 13s	remaining: 2.8s
1445:	learn: 0.0064391	total: 1m 13s	remaining: 2.75s
1446:	learn: 0.0064391	total: 1m 13

Create model (Dictionary with parameters and classifier)

In [7]:
rf_model = {
    'min_distance' : 0.01,
    'delta' : 0.007,
    'model' : clf_rf
}

xgb_model = {
    'min_distance' : 0.01,
    'delta' : 0.007,
    'model' : clf_xgb
}

cb_model = {
    'min_distance' : 0.01,
    'delta' : 0.007,
    'model' : clf_cb
}

Save models in */data/models/*

In [8]:
with open(os.path.join(os.path.dirname(os.path.abspath('..')), 'data', 'models', 'rf_model_user_try.pkl'), 'wb') as f:
    pkl.dump(rf_model, f)

with open(os.path.join(os.path.dirname(os.path.abspath('..')), 'data', 'models', 'xgb_model_user_try.pkl'), 'wb') as f:
    pkl.dump(xgb_model, f)

with open(os.path.join(os.path.dirname(os.path.abspath('..')), 'data', 'models', 'cb_model_user_try.pkl'), 'wb') as f:
    pkl.dump(cb_model, f)
